<a href="https://colab.research.google.com/github/hwon-n/cp2/blob/hyewon/CP2_NSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers

     |████████████████████████████████| 3.8 MB 4.1 MB/s 
     |████████████████████████████████| 6.5 MB 23.2 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
     |████████████████████████████████| 895 kB 41.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Softmax
from transformers import TFBertForNextSentencePrediction
from transformers import AutoTokenizer

In [3]:
# 모델과 tokenizer 준비

model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt = True)
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

# 동일 문장 여부 파악

In [4]:
# 임의의 문장 예측 (네이버 기사)

# 이어지는 문장
prompt1 = '정부는 약사 등의 '
next_sentence1 = '환자 발생으로 약국 영업이 중단될 경우 해당 지역의 보건 서비스에 심각한 영향이 우려돼 이번 지침을 마련했다고 설명했다.'

# 이어지지 않는 문장
prompt2 = '튜브 겉면에도 복합 재질을 의미하는 '
next_sentence2 = '연기를 할 수 없어 이같은 방법을 선택한 것으로 보인다.'

In [5]:
def same_sentence(prompt, next):
  encoding = tokenizer(prompt, next, return_tensors='tf')
  logits = model(encoding['input_ids'], token_type_ids = encoding['token_type_ids'])[0]
  softmax = Softmax()
  probs = softmax(logits)
  res = tf.math.argmax(probs, axis=-1).numpy()

  return res

In [6]:
res1 = same_sentence(prompt1, next_sentence1)

if res1 == 0:
  print('두 문장은 이어지는 문장입니다.')
else:
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지는 문장입니다.


In [7]:
res2 = same_sentence(prompt2, next_sentence2)

if res2 == 0:
  print('두 문장은 이어지는 문장입니다.')
else: 
  print('두 문장은 이어지지 않는 문장입니다.')

두 문장은 이어지지 않는 문장입니다.


# 데이터셋 준비

In [14]:
import os

root = '/content/drive/MyDrive/news_class9x13000/'
categories = os.listdir(root)

dataset = []

for category in categories:
  files = os.listdir(root + category)
  for i, f in enumerate(files):
    fname = root + category + '/' + f
    file = open(fname, 'r', encoding='utf-8')
    strings = file.read()
    dataset.append([strings])
    file.close()
  

In [15]:
len(dataset)

117010

In [16]:
table = str.maketrans({
    '\'': '',
    '\"': '',
    '★': '',
    '▲': '',
    '┌': '',
    '├': '',
    '│': '',
    '└': '',
    '→': '',
    '‘': '',
    '■': ''

})

In [ ]:
process_dataset = []

for sentences in dataset:
  sentences = sentences[0]
  sentences = sentences.translate(table)
  sentences = sentences.split('\n')
  process_dataset.extend(sentences)

process_dataset

In [18]:
len(process_dataset)

4200445

In [19]:
with open(
    os.path.join(root, 'process_dataset'), mode = 'w', encoding = 'utf-8'
) as f:
  for data in process_dataset:
    f.write(data)

print('save the preprocessing data file')

save the preprocessing data file


## 데이터 만들기

    corpus = [prompt, next, isNext]

    for index in range(process_dataset):
      if len(corpus) % 2 == 0: 
        이어지는 문장
        corpus.append([process_dataset[index - 1], process_dataset[index], True])
      else: 
        이어지지 않는 문장
        randomIndex = randint(0, len(corpus)-1)
        corpus.append([process_dataset[randomIndex], process_dataset[index], False])





In [ ]:
import pandas as pd

split_sentence = pd.DataFrame(columns=['prompt', 'next', 'same'])
split_sentence.head()

,prompt,next,same


In [ ]:
from random import randint

# 문장 분리

index = 0

for sent in process_dataset:
  x = randint(0, len(sent)-1)
  split_sentence.loc[index] = [sent[:x], sent[x:], 1]
  index += 1

split_sentence

,prompt,next,same
0,기타 신동,도 이 대통령도 유튜브로 돈 벌 수 있다,1
1,한국 개인,사용자들이 전 세계를 상대로 수익을 얻을 기회다.,1
2,온라인 동영상 사이트 유튜브는 6일 오전 10시 30분 서울 역,삼동 구글코리아 본사에서 유튜브 3.0을 소개하는 기자간담회를 열었다.,1
3,,유튜브는 하루 조회수 30억 건에 이르는 세계 최대 온라인 동영상 사이트로 한국을 ...,1
4,"구글 검색, 지메일 등 다른 서비스들이 네이버, 다음과 같은 토종 포털에 밀려 한국...",1위를 고수해 왔다.,1
...,...,...,...
520,"그러면서 그는 국회에서 중소영세상인 살리기 의원 모임을 만들기로 했고, 중소상인과 ...",완벽한 제도가 되도록 힘을 모아야 할 것이라고 제시했다.,1
521,동네구멍가게는 대기업들이 하는 장사,아니다,1
522,이휘웅 경남유통상인연합회 회장은 인사말을 통해 대기업은 대기업답게 해야 될 사업이 ...,"장사로, 우리나라를 대표하는 대기업들이 하는 장사가 아니다고 말했다.",1
523,"손석형 경남도의원은 세계화, 글로벌 등으로 외국문화와 자본을 무조건 받아들이다 보니...","기 위한 준비와 정책, 지원이 필요한데 정부는 손을 놓고 있다고 비판했다.",1


# Fine tuning

1. 데이터셋준비 (완료)
2. 데이터 정제
3. 데이터 토큰화
4. 데이터 패딩 및 attention mask
5. train, val, test dataset
6. train, val, test를 torch tensor로 변환
7. TensorDataset으로 train input, mask, label 묶기
8. 모델 device를 `cuda:0`으로 설정
 (`model.to(device)` )
9. BertForSequenceClassification.from_pretrained => pretrained 된 bert 불러오기

  [간단한 예제](https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/)

10. 옵티마이저 설정
11. 스케줄러 설정
12. 랜덤 시드 고정
13. 모델 그래디언트 초기화
14. Epoch만큼 훈련 반복

참고 : [BERT 모델의 장점과 Fine-Tuning 흐름 정리](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=antler07&logNo=222066071573)